In [1]:
import openai
import os
from dotenv import load_dotenv
# loads .env file located in the current directory
load_dotenv() 

openai.api_key = os.getenv('OPENAI_API_KEY')

In [6]:
query="select vid, ctextid, createdby, Created, description, p.dCode, dense_rank() over (partition by vid,ctextid order by Created) as rn from cte3 cross apply openjson(info) with (dCode varchar(30) '$.dCode',Description varchar(30) '$.description') p), R1 as (select * from Q1 where rn = 1), R2 as (select * from Q1 where rn = 2), Q2 as (select coalesce(R1.vid, R2.vid) as vid, coalesce(R1.ctextid, R2.ctextid) as ctextid, R1.Description as Description1, R1.dCode as dCode1, R2.Description as Description2, R2.dCode as dCode2 from R1 full outer join R2 on R2.vid = R1.vid and R2.ctextid = R1.ctextid and R2.[Description] = R1.[Description]) select vid, ctextid, (select top(1) createdby from R1 where vid = t.vid and ctextid = t.ctextid) as codername, coalesce( string_agg(case when dCode1 <> dCode2 then dCode1 end, ', '), '') as correctedcode, coalesce( string_agg(case when dCode2 is null then dCode1 end, ', '), '') as deletedcode, coalesce( string_agg(case when dCode1 is null then dCode2 end, ', '), '') as addedcode from Q2 as t group by vid, ctextid order by vid"

In [ ]:
# wrap the query string
query_two = "select h.iixctrvjg, h.uhczvi, h.mzxxvnfxrwmhdv, case when h.afprnxna is null then cast('06/09/1965' as date) else h.afprnxna end as afprnxna, h.khrcb, h.rmiuotcyosovtexo, h.cmvnonskhggg, h.ymrkz, h.rxfbop, h.jumlejbrvjeh, (select ct.number from NexusSuite.dbo.ClaimTransactions ct with (nolock) where ct.IsDeleted=0 and ct.[Key]=h.mzxxvnfxrwmhdv) as llmhiebazfms, case when NOT ( select top 1 cp.PolicyTransactionKey from NexusSuite.dbo.ClaimPolicies cp with (nolock) where cp.IsDeleted = 0 and cp.mzxxvnfxrwmhdv = h.mzxxvnfxrwmhdv order by cp.[Key] desc ) is null then ( select top 1 pt.EffectiveDate from NexusSuite.dbo.PolicyTransactions pt with (nolock) where pt.IsDeleted = 0 and pt.[Key] = ( select top 1 cp.PolicyTransactionKey from NexusSuite.dbo.ClaimPolicies cp with (nolock) where cp.IsDeleted = 0 and cp.mzxxvnfxrwmhdv = h.mzxxvnfxrwmhdv order by cp.[Key] desc ) ) else ( select top 1 cp.PolicyDate from NexusSuite.dbo.ClaimPolicies cp with (nolock) where cp.IsDeleted = 0 and cp.mzxxvnfxrwmhdv = h.mzxxvnfxrwmhdv order by cp.[Key] desc ) end as cgumdtceaen, case when h.iixctrvjg = 'Loss Payments' then h.amount else 0 end bofavqlampcvx, case when h.iixctrvjg = 'Expense Payments' then h.amount else 0 end uputumbtmqjyolbw, case when h.iixctrvjg = 'Deductible Recoveries' then h.amount else 0 end ntcgwapznskrqtcvoghrg, case when h.iixctrvjg = 'Recoveries' then h.amount else 0 end jgvukysogr, case when h.iixctrvjg = 'Loss Reserves' then h.amount else 0 end iesyymzvcswej, case when h.iixctrvjg = 'Expense Reserves' then h.amount else 0 end rckwydvepayhvgnp, case when h.iixctrvjg = 'Prior Loss Reserves' then h.amount else 0 end nkkrbhcwchzqmttquzo, case when h.iixctrvjg = 'Prior Expense Reserves' then h.amount else 0 end lwaffwwvaegimvhgmtrkcc from ( select 'Loss Payments' as iixctrvjg, cp.[Key] as uhczvi, ct.[Key] as mzxxvnfxrwmhdv, ct.afprnxna, cp.khrcb, cp.rmiuotcyosovtexo, cp.Description as cmvnonskhggg, cp.ymrkz, cp.rxfbop, cp.Date as jumlejbrvjeh from NexusSuite.dbo.ClaimPayments cp with (nolock) inner join NexusSuite.dbo.ClaimCashTypes cct with (nolock) on cct.[Key] = cp.rmiuotcyosovtexo inner join NexusSuite.dbo.ClaimTransactions ct with (nolock)  on ct.[Key] = cp.mzxxvnfxrwmhdv where cp.IsDeleted = 0 and {% condition accounting_scope %} cp.khrcb {% endcondition %} and ( ( cp.khrcb = 'Underwriter' and  cct.ClaimAccountingClass = 'Loss') or ( cp.khrcb = 'Agent' and cct.ClaimAccountingClass in ( 'Loss','Deductible') ) ) --and cp.[Date] between @ReportStart and @ReportEnd and {% condition report_date %} cp.[Date] {% endcondition %} UNION ALL select 'Expense Payments' as iixctrvjg, cp.[Key] as uhczvi, ct.[Key] as mzxxvnfxrwmhdv, ct.afprnxna, cp.khrcb, cp.rmiuotcyosovtexo, cp.Description as cmvnonskhggg, cp.ymrkz, cp.rxfbop, cp.Date as jumlejbrvjeh from NexusSuite.dbo.ClaimPayments cp with (nolock) inner join NexusSuite.dbo.ClaimCashTypes cct with (nolock) on cct.[Key] = cp.rmiuotcyosovtexo inner join NexusSuite.dbo.ClaimTransactions ct with (nolock)  on ct.[Key] = cp.mzxxvnfxrwmhdv where cp.IsDeleted = 0 and {% condition accounting_scope %} cp.khrcb {% endcondition %} and cct.ClaimAccountingClass = 'Expense' and {% condition report_date %} cp.[Date] {% endcondition %} h"


In [6]:
to_sql="Snowflake"
from_sql="Transact-SQL"

In [3]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0): 
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message["content"]

In [5]:
prompt = f"""'Act as CODEX ("COding DEsign eXpert"), an expert coder with proficiency in SQL programming language.
You are proficient in Transact-SQL, MySQL, PL/SQL, PL/pgSQL, SQLite, and Snowflake SQL dialects, focusing on high-accuracy dialect-to-dialect conversions.
Your task is to convert a specific SQL script from one SQL dialect to another while maintaining the original script's functionality and integrity.
Before you start the conversion, your first task is removing square brackets and double quotes from column and table names.
The source SQL dialect is "{from_sql}," and the target SQL dialect is "{to_sql}." You aim to perform a precise SQL dialect conversion while addressing incompatibilities or differences.
Always follow the coding best practices by writing clean, modular code with proper security measures and leveraging design patterns.
Let''s think step by step. First, you will identify the differences between the {from_sql} and {to_sql} dialects.
You will address differences in data types and functions between the {from_sql} and {to_sql} dialects. Choose the most suitable alternative for data types or functions without a direct equivalent.
When converting from Transact-SQL to Snowflake, ensure that double quotes are not added to field names. 
Output results in SQL query format. Only return the converted sql query and do not include any other text or characters.
You will be provided with a SQL query ```{query}``` in backticks. Convert the SQL dialect of ```{query}``` from "{from_sql}" to "{to_sql}" while ensuring the highest level of accuracy in maintaining the original functionality:
'"""
response = get_completion(prompt)
print(response)

SELECT [FirstName], [LastName], [Email]
FROM [Users]
WHERE [City] = 'New York'
ORDER BY [LastName] ASC;'

SELECT "FirstName", "LastName", "Email"
FROM Users
WHERE "City" = 'New York'
ORDER BY "LastName" ASC;


In [ ]:
def sql_dialectify(from_sql, to_sql, original_sql, mask_fields=False, identifiers=None):
    # If mask_fields is True, mask the fields using the provided sql_masking function
    if mask_fields and identifiers:
        masked_sql, word_map = sql_masking(identifiers, original_sql)
    else:
        masked_sql = original_sql

    completion = openai.ChatCompletion.create(
        #model="gpt-4",
        model="gpt-3.5-turbo",
        temperature=0.2,
        messages=[
           {"role": "system", "content": 'Act as CODEX ("COding DEsign eXpert"), an expert coder with proficiency in SQL programming language.'},
            {"role": "system", "content": 'You are proficient in Transact-SQL, MySQL, PL/SQL, PL/pgSQL, SQLite, and Snowflake SQL dialects, with a focus on high accuracy dialect to dialect conversions.'},
            {"role": "system", "content": 'Your task is to convert a specific SQL script from one SQL dialect to another SQL dialect while maintaining the functionality and integrity of the original script.'},
            {"role": "system", "content": f'The source SQL dialect is "{from_sql}", and the target SQL dialect is "{to_sql}". Your goal is to perform a precise SQL dialect conversion while addressing any incompatibilities or differences.'},
            {"role": "system", "content": 'Always follow the coding best practices by writing clean, modular code with proper security measures and leveraging design patterns.'},
            {"role": "system", "content": f'Let''s think step by step. First, you will identify the differences between the {from_sql} and {to_sql} dialects. Then, you will convert the SQL code from {from_sql} to {to_sql}.'},
            {"role": "system", "content": f'You will identify and address differences in data types and functions between the {from_sql} and {to_sql} dialects. For data types or functions without a direct equivalent, choose the most suitable alternative'},
            # {"role": "system", "content": 'You will return your answers in two sections. In the first section you will return the converted sql query in a code block and you will add a semi colon (;) at the end of the query if it was not inputted. You will title this section as "\n # Converted SQL: ".'},
            # {"role": "system", "content": 'In the second section you will return any comments and the explanations of the changes in bulled points. You will title this section as "\n List of changes: ".'},
            {"role": "system", "content": 'Do not add double quotes around the table names.'}, 
            {"role": "system", "content": 'Output results in a sql query format.'},
            {"role": "user", "content": f'Convert the SQL dialect of the following SQL query in backticks from "{from_sql}" to "{to_sql}" while ensuring the highest level of accuracy in maintaining the original functionality: ```\n\n{masked_sql}```\n\n'}
        
        ]
    )

    converted_sql = completion.choices[0].message.content

    # If mask_fields is True, replace the masked words with the original identifiers
    if mask_fields and identifiers:
        for original, masked in word_map.items():
            converted_sql = converted_sql.replace(masked, original)

    return converted_sql

In [ ]:
import sqlparse
# print(sql_dialectify(to_sql, query))
print(sqlparse.format(sql_dialectify(to_sql, query_two), reindent=True, keyword_case='upper'))

In [7]:
query="""select r.transaction_key,
    isnull(sum(r.net_premium),0) as net_premium,
    isnull(sum(r.payment_amount),0) as payment_amount,
    isnull(sum(r.net_premium),0) - isnull(sum(r.payment_amount),0) as balance_due
    from (
    select
    p.TransactionKey as transaction_key,
    isnull(sum(p.NetPremium),0) as net_premium,
    0 as payment_amount
    from NexusSuite.dbo.policies p with (nolock)
    INNER JOIN NexusSuite.dbo.policytransactions pt with (nolock) on pt.[Key] = p.TransactionKey
    INNER JOIN NexusSuite.dbo.companies c with (nolock) on c.[Key] = pt.CompanyKey and c.CompanyType in (0,1,2,3)
    where
    p.IsDeleted = 0 and p.IsVoided = 0 and pt.IsDeleted=0 and c.[key] > 1
     and p.CreatedDate between cast('01/01/1900' as datetime) and cast(getdate()-1 as datetime)
     group by p.TransactionKey

    union all

    --------------------------------------------------------------------------------------------------------------------
    --bof: get endorsements
    --------------------------------------------------------------------------------------------------------------------
    select
    e.TransactionKey,
    isnull(sum(e.NetPremium),0) as NetPremium,
    0 as payment_amount
    from NexusSuite.dbo.endorsements e with (nolock)
    INNER JOIN NexusSuite.dbo.policytransactions pt with (nolock) on pt.[Key] = e.TransactionKey
    INNER JOIN NexusSuite.dbo.companies c with (nolock) on c.[Key] = pt.CompanyKey and c.CompanyType in (0,1,2,3)
    where e.IsDeleted = 0 and e.IsVoided = 0 and pt.IsDeleted=0 and c.[key] > 1
    and e.CreatedDate between cast('01/01/1900' as datetime) and cast(getdate()-1 as datetime)
    group by e.TransactionKey

    union all

    --------------------------------------------------------------------------------------------------------------------
    --bof: get cpls
    --------------------------------------------------------------------------------------------------------------------
    select
    c.TransactionKey,
    isnull(sum(c.NetPremium),0) as NetPremium,
    0 as payment_amount
    from NexusSuite.dbo.cpls c with (nolock)
    INNER JOIN NexusSuite.dbo.policytransactions pt with (nolock) on pt.[Key] = c.TransactionKey
    INNER JOIN NexusSuite.dbo.companies co with (nolock) on co.[Key] = pt.CompanyKey and co.CompanyType in (0,1,2,3)
    where
    c.IsDeleted = 0 and c.IsVoided = 0 and pt.IsDeleted=0 and co.[key] > 1
    and c.CreatedDate between cast('01/01/1900' as datetime) and cast(getdate()-1 as datetime)
    group by c.TransactionKey

    union all

    --------------------------------------------------------------------------------------------------------------------
    --bof: get adjustments
    --------------------------------------------------------------------------------------------------------------------
    select
    a.TransactionKey,
    isnull(sum(a.NetPremium),0) as NetPremium,
    0 as payment_amount
    from NexusSuite.dbo.Adjustments a with (nolock)
    INNER JOIN NexusSuite.dbo.policytransactions pt with (nolock) on pt.[Key] = a.TransactionKey
    INNER JOIN NexusSuite.dbo.companies c with (nolock) on c.[Key] = pt.CompanyKey and c.CompanyType in (0,1,2,3)
    where a.IsDeleted = 0 and pt.IsDeleted=0 and c.[key] > 1
    and a.[CreatedDate] between cast('01/01/1900' as datetime) and cast(getdate()-1 as datetime)
    group by a.TransactionKey

    union all
    select
    f.TransactionKey, isnull(sum(f.Amount),0) as NetPremium, 0 as payment_amount from NexusSuite.dbo.fees f with (nolock) INNER join NexusSuite.dbo.PolicyTransactions pt with (nolock) on pt.[key] = f.TransactionKey INNER JOIN NexusSuite.dbo.companies c with (nolock) on c.[Key] = pt.CompanyKey and c.CompanyType in (0,1,2,3) where f.IsDeleted = 0 and pt.IsDeleted=0 and c.[key] > 1 and f.CreatedDate between cast('01/01/1900' as datetime) and cast(getdate()-1 as datetime) group by f.TransactionKey union all select re.TransactionKey, isnull(sum(re.AgentReceivable),0) as NetPremium, 0 as payment_amountfrom NexusSuite.dbo.Reinsurance re with (nolock) INNER JOIN NexusSuite.dbo.policytransactions pt with (nolock) on pt.[Key] = re.TransactionKey INNER JOIN NexusSuite.dbo.companies c with (nolock) on c.[Key] = pt.CompanyKey and c.CompanyType in (0,1,2,3) where re.IsDeleted = 0 and pt.IsDeleted=0 and c.[key] > 1 
    and re.CreatedDate between cast('01/01/1900' as datetime) and cast(getdate()-1 as datetime) group by re.TransactionKey
    """

to_sql="Snowflake"
from_sql="Transact-SQL"

In [22]:
def sql_dialectify(from_sql, to_sql, original_sql, model_choice="gpt-3.5-turbo", temperature=0, mask_fields=False, identifiers=None):
    # If mask_fields is True, mask the fields using the provided sql_masking function
   
    masked_sql = original_sql

    completion = openai.ChatCompletion.create(
        model=model_choice,
        temperature=temperature,
        messages=[
           {"role": "system", "content": 'Act as CODEX ("COding DEsign eXpert"), an expert coder with proficiency in SQL programming language.'},
            {"role": "system", "content": 'You are proficient in Transact-SQL, MySQL, PL/SQL, PL/pgSQL, SQLite, and Snowflake SQL dialects, with a focus on high accuracy dialect to dialect conversions.'},
            {"role": "system", "content": 'Your task is to convert a specific SQL script from one SQL dialect to another SQL dialect while maintaining the functionality and integrity of the original script.'},
            {"role": "system", "content": f'The source SQL dialect is "{from_sql}", and the target SQL dialect is "{to_sql}". Your goal is to perform a precise SQL dialect conversion while addressing any incompatibilities or differences.'},
            {"role": "system", "content": 'Always follow the coding best practices by writing clean, modular code with proper security measures and leveraging design patterns.'},
            {"role": "system", "content": f'Let''s think step by step. First, you will identify the differences between the {from_sql} and {to_sql} dialects.'},
            {"role": "system", "content": f'You will address differences in data types and functions between the {from_sql} and {to_sql} dialects. For data types or functions without a direct equivalent, choose the most suitable alternative'},
            {"role": "system", "content": 'Output results in SQL query format. Only return the converted sql query and do not include any other text or characters.'},
            {"role": "user", "content": f'You will be provided with a SQL query in backticks. Convert the SQL dialect of that SQL query in backticks from "{from_sql}" to "{to_sql}" while ensuring the highest level of accuracy in maintaining the original functionality: ```{masked_sql}```'},
            {"role": "system", "content": 'Check if the output replaced square brackets with double quotes. If so, remove the double quotes before returning the output. Such as return pt.Key instead of pt."Key"'}, 


        ]
    )

    converted_sql = completion.choices[0].message.content

    return converted_sql

In [23]:
import sqlparse
print(sql_dialectify(from_sql, to_sql, query))


Here's the Snowflake SQL equivalent of the given Transact-SQL query:

```
select r.transaction_key,
    coalesce(sum(r.net_premium),0) as net_premium,
    coalesce(sum(r.payment_amount),0) as payment_amount,
    coalesce(sum(r.net_premium),0) - coalesce(sum(r.payment_amount),0) as balance_due
    from (
    select
    p.TransactionKey as transaction_key,
    coalesce(sum(p.NetPremium),0) as net_premium,
    0 as payment_amount
    from NexusSuite.dbo.policies p 
    INNER JOIN NexusSuite.dbo.policytransactions pt on pt."Key" = p.TransactionKey
    INNER JOIN NexusSuite.dbo.companies c on c."Key" = pt.CompanyKey and c.CompanyType in (0,1,2,3)
    where
    p.IsDeleted = 0 and p.IsVoided = 0 and pt.IsDeleted=0 and c."key" > 1
     and p.CreatedDate between to_date('01/01/1900', 'MM/DD/YYYY') and to_date(current_date()-1)
     group by p.TransactionKey

    union all

    --------------------------------------------------------------------------------------------------------------------
 